In [1]:
import os
import subprocess
import mlflow
import logging

In [3]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def run_command(command):
    """Run a shell command and return the output"""
    result = subprocess.run(command, shell=True, check=True, text=True, capture_output=True)
    return result.stdout.strip()

def main():
    try:
        # 1. Add the DVC lock file
        run_command("git add *.dvc.lock")
        
        # 2. Commit the DVC lock file
        commit_message = "Update DVC lock file"
        run_command(f'git commit -m "{commit_message}"')
        
        # 3. Push changes to GitHub
        run_command("git push origin main")  # Adjust branch name if needed
        
        # 4. Get the commit ID
        commit_id = run_command("git rev-parse HEAD")
        logger.info(f"Commit ID: {commit_id}")
        
        # 5. Perform DVC checkout
        run_command("dvc checkout")
        
        # 6. Log the commit ID to MLflow
        mlflow.set_tracking_uri("http://localhost:5000")  # Adjust as needed
        mlflow.set_experiment("dvc_github_integration")
        
        with mlflow.start_run():
            mlflow.log_param("commit_id", commit_id)
            logger.info("Logged commit ID to MLflow")
        
        logger.info("All operations completed successfully")
    
    except subprocess.CalledProcessError as e:
        logger.error(f"Error executing command: {e.cmd}")
        logger.error(f"Error output: {e.stderr}")
    except Exception as e:
        logger.error(f"An error occurred: {str(e)}")

In [4]:
if __name__ == "__main__":
    main()

ERROR:__main__:Error executing command: git add *.dvc.lock
ERROR:__main__:Error output: /bin/sh: git: command not found

